<a href="https://colab.research.google.com/github/BraneXZ/CNN-down-sampled-imagenet/blob/master/down_sampled_imagenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import tensorflow_datasets as tfds
import numpy as np
import PIL.Image as Image
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow import keras
import math

tf.enable_eager_execution()

In [0]:
validate, info = tfds.load("downsampled_imagenet/64x64", data_dir="/content/gdrive/My Drive", split="validation", with_info=True)
train = tfds.load("downsampled_imagenet/64x64", data_dir="/content/gdrive/My Drive", split="train")

In [0]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/2" #@param {type:"string"}

In [0]:
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [0]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

In [0]:
validate_batch = validate.batch(128)
validate_labels = []
for batch in validate_batch:
  resized_images = tf.image.resize_images(batch['image'], (224, 224))
  image_data = image_generator.flow(resized_images/255)
  result = classifier.predict(image_data.x)
  for index, image in enumerate(resized_images/255):
#     print('{},'.format(np.argmax(result[index])))
    validate_labels.append(np.argmax(result[index]))
#     validate_with_labels['image'].append(image)
#     plt.figure()
#     plt.imshow(image)
#     plt.title(imagenet_labels[np.argmax(result[index], axis=-1)])
#     plt.show()

In [0]:
train_batch = train.batch(128)
train_labels = []
batch_num = 1
for batch in train_batch:
  resized_images = tf.image.resize_images(batch['image'], (224, 224))
  image_data = image_generator.flow(resized_images/255)
  result = classifier.predict(image_data.x)
  print("Processing batch {}".format(batch_num))
  batch_num += 1
  for index, image in enumerate(resized_images/255):
#     print('{},'.format(np.argmax(result[index])))
    train_labels.append(np.argmax(result[index]))
#     validate_with_labels['image'].append(image)
#     plt.figure()
#     plt.imshow(image)
#     plt.title(imagenet_labels[np.argmax(result[index], axis=-1)])
#     plt.show()

In [0]:
# create and compile model
def create_model(level_0_repeats, level_1_repeats, level_2_repeats):

    # encoder - input
    model_input = keras.Input(shape=(64, 64, 3), name='image')
    x           = model_input
    
    # encoder - level 0
    for n0 in range(level_0_repeats):
        x = keras.layers.Conv2D(32, 3, strides=1, padding='same', activation='relu', use_bias=True)(x)
#         x = keras.layers.Conv2D(32, 3, strides=1, padding='same', activation=None, use_bias=False)(x)
#         x = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x)
#         x = keras.layers.ReLU()(x)
    x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)

    # encoder - level 1
    for n1 in range(level_1_repeats):
        x = keras.layers.Conv2D(64, 3, strides=1, padding='same', activation='relu', use_bias=True)(x)
#         x = keras.layers.Conv2D(64, 3, strides=1, padding='same', activation=None, use_bias=False)(x)
#         x = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x)
#         x = keras.layers.ReLU()(x)
    x = keras.layers.MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
        
    # encoder - level 2
    for n2 in range(level_2_repeats):
      x = keras.layers.Conv2D(128, 3, strides=1, padding='same', activation='relu', use_bias=True)(x)
#         x = keras.layers.Conv2D(128, 3, strides=1, padding='same', activation=None, use_bias=False)(x)
#         x = keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True)(x)
#         x = keras.layers.ReLU()(x)

    # encoder - output
    encoder_output = x

    # decoder
    y              = keras.layers.GlobalAveragePooling2D()(encoder_output)
    decoder_output = keras.layers.Dense(1000, activation='softmax')(y)
    
    # forward path
    model = keras.Model(inputs=model_input, outputs=decoder_output, name='down_sampled_imagenet_model')

    # loss, backward path (implicit) and weight update
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    # return model
    return model


In [0]:
# create and compile model
model = create_model(3, 3, 3)

# model description and figure
model.summary()

Model: "down_sampled_imagenet_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image (InputLayer)           [(None, 64, 64, 3)]       0         
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 64, 64, 32)        896       
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 64, 64, 32)        9248      
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 64, 64, 32)        9248      
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_67 (Conv2D)           (None, 32,

In [0]:
# training
TRAINING_BATCH_SIZE      = 32
TRAINING_SHUFFLE_BUFFER  = 5000
TRAINING_LR_MAX          = 0.001
# TRAINING_LR_SCALE        = 0.1
# TRAINING_LR_EPOCHS       = 2
TRAINING_LR_INIT_SCALE   = 0.01
TRAINING_LR_INIT_EPOCHS  = 5
TRAINING_LR_FINAL_SCALE  = 0.01
TRAINING_LR_FINAL_EPOCHS = 25

# training (derived)
TRAINING_NUM_EPOCHS = TRAINING_LR_INIT_EPOCHS + TRAINING_LR_FINAL_EPOCHS
TRAINING_LR_INIT    = TRAINING_LR_MAX*TRAINING_LR_INIT_SCALE
TRAINING_LR_FINAL   = TRAINING_LR_MAX*TRAINING_LR_FINAL_SCALE

# saving
SAVE_MODEL_PATH = '/content/gdrive/My Drive/model'
!mkdir -p "$SAVE_MODEL_PATH"

In [0]:
# learning rate schedule
def lr_schedule(epoch):

    # staircase
    # lr = TRAINING_LR_MAX*math.pow(TRAINING_LR_SCALE, math.floor(epoch/TRAINING_LR_EPOCHS))

    # linear warmup followed by cosine decay
    if epoch < TRAINING_LR_INIT_EPOCHS:
        lr = (TRAINING_LR_MAX - TRAINING_LR_INIT)*(float(epoch)/TRAINING_LR_INIT_EPOCHS) + TRAINING_LR_INIT
    else:
        lr = (TRAINING_LR_MAX - TRAINING_LR_FINAL)*max(0.0, math.cos(((float(epoch) - TRAINING_LR_INIT_EPOCHS)/(TRAINING_LR_FINAL_EPOCHS - 1.0))*(math.pi/2.0))) + TRAINING_LR_FINAL

    # debug - learning rate display
    # print(epoch)
    # print(lr)

    return lr

# plot training accuracy and loss curves
def plot_training_curves(history):

    # training and validation data accuracy
    acc     = history.history['accuracy']
    val_acc = history.history['val_accuracy']

    # training and validation data loss
    loss     = history.history['loss']
    val_loss = history.history['val_loss']

    # plot accuracy
    plt.figure(figsize=(8, 8))
    plt.subplot(2, 1, 1)
    plt.plot(acc, label='Training Accuracy')
    plt.plot(val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.ylabel('Accuracy')
    plt.ylim([min(plt.ylim()), 1])
    plt.title('Training and Validation Accuracy')

    # plot loss
    plt.subplot(2, 1, 2)
    plt.plot(loss, label='Training Loss')
    plt.plot(val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.ylabel('Cross Entropy')
    plt.ylim([0, 2.0])
    plt.title('Training and Validation Loss')
    plt.xlabel('epoch')
    plt.show()

# callbacks (learning rate schedule, model checkpointing during training)
callbacks = [keras.callbacks.LearningRateScheduler(lr_schedule),
             keras.callbacks.ModelCheckpoint(filepath=SAVE_MODEL_PATH+'model_{epoch}.h5', save_best_only=True, monitor='val_loss', verbose=1)]


In [0]:
# with open('train_label.txt', 'w') as filehandle:
#     for items in train_labels:
#         filehandle.write('%s\n' % items)

In [0]:
# with open('validate_label.txt', 'w') as filehandle:
#     for items in validate_labels:
#         filehandle.write('%s\n' % items)

In [0]:
train_label = []
with open('/content/gdrive/My Drive/downsampled_imagenet/64x64/1.0.0/train_label.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        currentPlace = line[:-1]

        # add item to the list
        train_label.append(int(currentPlace))

In [0]:
validate_label = []
with open('/content/gdrive/My Drive/downsampled_imagenet/64x64/1.0.0/validate_label.txt', 'r') as filehandle:
    for line in filehandle:
        # remove linebreak which is the last character of the string
        currentPlace = line[:-1]

        # add item to the list
        validate_label.append(int(currentPlace))

In [0]:
# def tfdata_generator(images, labels, is_training, batch_size=128):
#   '''Construct a data generator using `tf.Dataset`. '''

#   def map_fn(image, label):
#     x = tf.reshape(tf.cast(image, tf.float32), (64, 64, 1))
#     y = tf.one_hot(tf.cast(label, tf.uint8), 1000)
#     return x, y

#   dataset = tf.data.Dataset.from_tensor_slices(images)

#   if is_training:
#     dataset = dataset.shuffle(1000)  # depends on sample size
#   dataset = dataset.map(map_fn)
#   dataset = dataset.batch(batch_size)
#   dataset = dataset.repeat()
#   dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)

#   return dataset

In [0]:
train_label = np.asarray(train_label)

In [0]:
def batching_label(iterable, n=128):
    l = len(iterable)
    for ndx in range(0, l, n):
        label = iterable[ndx:min(ndx + n, l)]
        yield label
train_label = batching_label(train_label)

In [0]:
def combine(images, labels):
  for image, label in zip(images, labels):
    yield (tf.divide(image['image'],64), tf.constant(label))

In [0]:
train_batch = train.batch(128)
train_combined = combine(train_batch, train_label)

In [0]:
for i in train_combined:
  print(i)
  break

(<tf.Tensor: id=49875, shape=(128, 64, 64, 3), dtype=float32, numpy=
array([[[[2.46875 , 2.453125, 2.453125],
         [1.875   , 1.875   , 1.875   ],
         [3.984375, 3.984375, 3.984375],
         ...,
         [3.59375 , 3.59375 , 3.59375 ],
         [1.984375, 1.9375  , 1.953125],
         [1.75    , 1.734375, 1.734375]],

        [[2.46875 , 2.453125, 2.453125],
         [1.875   , 1.875   , 1.875   ],
         [3.984375, 3.984375, 3.984375],
         ...,
         [3.984375, 3.984375, 3.984375],
         [1.96875 , 1.921875, 1.9375  ],
         [1.78125 , 1.734375, 1.75    ]],

        [[2.46875 , 2.453125, 2.453125],
         [1.875   , 1.875   , 1.875   ],
         [3.984375, 3.984375, 3.984375],
         ...,
         [3.375   , 3.375   , 3.375   ],
         [2.046875, 2.      , 2.015625],
         [1.765625, 1.734375, 1.734375]],

        ...,

        [[2.46875 , 2.453125, 2.453125],
         [1.875   , 1.875   , 1.875   ],
         [3.984375, 3.984375, 3.984375],
        

In [0]:
# training
initial_epoch_num = 0
history           = model.fit_generator(train_combined, steps_per_epoch=1281149//128, epochs=TRAINING_NUM_EPOCHS, verbose=1, callbacks=callbacks, initial_epoch=initial_epoch_num)

# example of restarting training after a crash from the last saved checkpoint
# model             = create_model(MODEL_LEVEL_0_REPEATS, MODEL_LEVEL_1_REPEATS, MODEL_LEVEL_2_REPEATS)
# model.load_weights(SAVE_MODEL_PATH+'model_X.h5') # replace X with the last saved checkpoint number
# initial_epoch_num = X                            # replace X with the last saved checkpoint number
# history           = model.fit(x=dataset_train, epochs=TRAINING_NUM_EPOCHS, verbose=1, callbacks=callbacks, validation_data=dataset_test, initial_epoch=initial_epoch_num)

# plot accuracy and loss curves
plot_training_curves(history)

Epoch 1/30
 8655/10008 [========================>.....] - ETA: 1:31 - loss: nan - acc: 1.8053e-06WARNING:tensorflow:Your dataset iterator ran out of data; interrupting training. Make sure that your iterator can generate at least `steps_per_epoch * epochs` batches (in this case, 300240 batches). You may need touse the repeat() function when building your dataset.


KeyError: ignored

In [0]:
test, info = tfds.load("mnist", split="train", with_info=True)

In [0]:
test = test.batch(32)